# GCD own algorithm by lambda-calculus lib
[GCD detailed script](https://www.mathcha.io/editor/dGXnWtP8twWsGPdy2Ycl5ly7xhd9Xy8ptKleZqr)

In [1]:
import time

from lambda_calculus import Variable as Var
from lambda_calculus import Abstraction as Lambda
from lambda_calculus import Application as App

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor
from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

In [2]:
def multi_app_term(term_0, term_1, *terms):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term

In [3]:
def leq_term():
    m, n = Var("mmm"), Var("nnn")
    return Lambda("mmm", Lambda("nnn", App(
        arithmetic.ISZERO,
        App(App(arithmetic.SUBTRACT, m), n)
    )))

In [4]:
def eq_term():
    mm, nn = Var("mm"), Var("nn")
    return Lambda("mm", Lambda("nn", multi_app_term(
        logic.AND,
        multi_app_term(leq_term(), mm, nn),
        multi_app_term(leq_term(), nn, mm)
    )))

In [5]:
def gcd_term_v0():
    n, m, f = Var("n"), Var("m"), Var("f")

    swap_line = multi_app_term(
        logic.IF_THEN_ELSE,
        multi_app_term(leq_term(), n, m),
        multi_app_term(f, m, multi_app_term(arithmetic.SUBTRACT, n, m)),
        multi_app_term(f, n, multi_app_term(arithmetic.SUBTRACT, m, n)),
    )

    eq_nm_line = multi_app_term(logic.IF_THEN_ELSE, multi_app_term(eq_term(), n, m), n, swap_line)
    zero_check = multi_app_term(logic.OR, App(arithmetic.ISZERO, n), App(arithmetic.ISZERO, m))

    inner_app = multi_app_term(logic.IF_THEN_ELSE, zero_check, arithmetic.number(1), eq_nm_line)
    inner_lambda = Lambda("f", Lambda("n", Lambda("m", inner_app)))
    return App(combinators.Y, inner_lambda)


def test_gcd_v0(var_0=0, var_1=0):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v0(), arithmetic.number(var_0), arithmetic.number(var_1))
    res_term = BetaNormalisingVisitor().skip_intermediate(term_)

    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [6]:
test_gcd_v0(0, 0)

(λf.(λx.(f x)))
time normalization: 0.0009999275207519531s


In [7]:
test_gcd_v0(0, 1)

(λf.(λx.(f x)))
time normalization: 0.0010004043579101562s


In [8]:
test_gcd_v0(1, 0)

(λf.(λx.(f x)))
time normalization: 0.0010004043579101562s


In [9]:
test_gcd_v0(10, 5)

(λf.(λx.(f x)))
time normalization: 0.016004323959350586s


In [10]:
test_gcd_v0(5, 10)

(λf.(λx.(f x)))
time normalization: 0.016003131866455078s


# test the v00 realization

In [11]:
def gcd_term_v00():
    n, m, f = "n", "m", "f"
    n_, m_, f_ = Var(n), Var(m), Var(f)
    sub_m_n = App(App(arithmetic.SUBTRACT, m_), n_)
    sub_n_m = App(App(arithmetic.SUBTRACT, n_), m_)
    iszero_n = App(arithmetic.ISZERO, n_)
    iszero_m = App(arithmetic.ISZERO, m_)
    f_n_sub = App(App(f_, n_), sub_m_n)
    f_m_sub = App(App(f_, m_), sub_n_m)
    le_n_m = App(App(leq_term(), n_), m_)
    eq_n_m = App(App(eq_term(), n_), m_)
    or_is_zero = App(logic.OR, App(iszero_n, iszero_m))
    ite_le_sub_sub = App(App(App(logic.IF_THEN_ELSE, le_n_m), f_n_sub), f_m_sub)
    ite_eq_ite = App(App(App(logic.IF_THEN_ELSE, eq_n_m), n_), ite_le_sub_sub)
    or_is_ite = App(App(or_is_zero, arithmetic.number(0)), ite_eq_ite)
    inner_app = App(logic.IF_THEN_ELSE, or_is_ite)
    inner_lambda = Lambda(f, Lambda(n, Lambda(m, inner_app)))
    return App(combinators.Y, inner_lambda)


def test_gcd_v00(var_0=0, var_1=0):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v00(), arithmetic.number(var_0), arithmetic.number(var_1))
    res_term = BetaNormalisingVisitor().skip_intermediate(term_)

    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [12]:
test_gcd_v00(0, 0)

(λa.(λb.b))
time normalization: 0.0020003318786621094s


In [13]:
test_gcd_v00(1, 0)

(λa.(λb.(a b)))
time normalization: 0.002000570297241211s


In [14]:
test_gcd_v00(0, 1)

KeyboardInterrupt: 

In [15]:
test_gcd_v00(6, 3)

(λa.(λb.(a b)))
time normalization: 0.0010004043579101562s


In [16]:
test_gcd_v00(3, 6)

(λa.(λb.(a b)))
time normalization: 0.002000570297241211s


In [17]:
test_gcd_v00(10, 10)

(λa.(λb.(a b)))
time normalization: 0.0019998550415039062s


# test the v01 realization

In [20]:
def gcd_term_v01():
    n, m, f = "n", "m", "f"
    n_, m_, f_ = Var(n), Var(m), Var(f)
    sub_m_n = App(App(arithmetic.SUBTRACT, m_), n_)
    sub_n_m = App(App(arithmetic.SUBTRACT, n_), m_)
    iszero_n = App(arithmetic.ISZERO, n_)
    iszero_m = App(arithmetic.ISZERO, m_)
    f_n_sub = App(App(f_, n_), sub_m_n)
    f_m_sub = App(App(f_, m_), sub_n_m)
    le_n_m = App(App(leq_term(), n_), m_)
    eq_n_m = App(App(eq_term(), n_), m_)
    or_is_zero = App(logic.OR, App(iszero_n, iszero_m))
    ite_le_sub_sub = App(App(App(logic.IF_THEN_ELSE, le_n_m), f_n_sub), f_m_sub)
    ite_eq_ite = App(App(App(logic.IF_THEN_ELSE, eq_n_m), n_), ite_le_sub_sub)
    # or_is_ite = App(App(or_is_zero, n0_term()), ite_eq_ite)
    # inner_app = App(ite_term(), or_is_ite)
    inner_app = App(App(App(logic.IF_THEN_ELSE, or_is_zero), arithmetic.number(0)), ite_eq_ite)
    inner_lambda = Lambda(f, Lambda(n, Lambda(m, inner_app)))
    return App(combinators.Y, inner_lambda)


def test_gcd_v01(var_0=0, var_1=0):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v01(), arithmetic.number(var_0), arithmetic.number(var_1))
    res_term = BetaNormalisingVisitor().skip_intermediate(term_)

    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [23]:
test_gcd_v01(0, 0)

(λf.(λx.x))
time normalization: 0.0010006427764892578s


In [24]:
test_gcd_v01(1, 0)

(λx.x)
time normalization: 0.0020008087158203125s


In [25]:
test_gcd_v01(0, 1)

KeyboardInterrupt: 

In [26]:
test_gcd_v01(1, 1)

(λx.x)
time normalization: 0.0010008811950683594s


In [27]:
test_gcd_v01(1, 2)

(λx.x)
time normalization: 0.0020003318786621094s


In [28]:
test_gcd_v01(3, 3)

(λx.x)
time normalization: 0.0010008811950683594s
